In [1]:
%load_ext autoreload
%autoreload 2

import scipy
import numpy as np
import tensorflow as tf
import keras
import tensorflow_probability as tfp



2024-06-14 08:51:37.892300: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-14 08:51:37.956302: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-14 08:51:37.956337: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-14 08:51:37.957623: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-14 08:51:37.965857: I tensorflow/core/platform/cpu_feature_guar

In [2]:
    # Distribution Settings
    scale = 1
    mean = 0

    normal_dist = scipy.stats.norm(loc=mean, scale=scale)

    # Number of independent datapoints 
    T = 1000

    seed=360
    rng = np.random.RandomState(seed)
    y_T = normal_dist.rvs(size=T, random_state=rng)

    # create keras model
    inputs = keras.Input(shape=0)
    # a dense layer with an empty input will just be an intercept term
    intercept_layer = keras.layers.Dense(1, activation = None)
    # Create a normal distribution with a mean equal to the input 
    distribution_layer = tfp.layers.DistributionLambda(lambda t: tfp.distributions.Normal(loc=t, scale=1))
    predicted_mean = intercept_layer(inputs)
    distribution = distribution_layer(predicted_mean)
    model = keras.Model(inputs=inputs, outputs=distribution)

2024-06-14 08:52:32.179104: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-06-14 08:52:32.179151: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: s1cmp008.pax.tufts.edu
2024-06-14 08:52:32.179156: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: s1cmp008.pax.tufts.edu
2024-06-14 08:52:32.179246: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 535.129.3
2024-06-14 08:52:32.179272: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 535.129.3
2024-06-14 08:52:32.179278: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 535.129.3


In [3]:
    # Create an empty input so we can do a forward pass
    x_TF = np.zeros(shape=(T,0))


In [7]:
    # number of score function trick samples
    N=50
    with tf.GradientTape() as tape:
        model_distribution = model(x_TF)
        sample_y_NT = model_distribution.sample(N)
        stopped_samples = tf.stop_gradient(sample_y_NT)
        sample_log_probs_NT = model_distribution.log_prob(stopped_samples)

    # use jacobian to get gradient of each sample
    jacobian_params_NT = tape.jacobian(sample_log_probs_NT, model.trainable_weights)

In [18]:
jacobian_params_NT

[<tf.Tensor: shape=(5, 1000, 1, 0, 1), dtype=float32, numpy=array([], shape=(5, 1000, 1, 0, 1), dtype=float32)>,
 <tf.Tensor: shape=(5, 1000, 1, 1), dtype=float32, numpy=
 array([[[[1.]],
 
         [[1.]],
 
         [[1.]],
 
         ...,
 
         [[1.]],
 
         [[1.]],
 
         [[1.]]],
 
 
        [[[1.]],
 
         [[1.]],
 
         [[1.]],
 
         ...,
 
         [[1.]],
 
         [[1.]],
 
         [[1.]]],
 
 
        [[[1.]],
 
         [[1.]],
 
         [[1.]],
 
         ...,
 
         [[1.]],
 
         [[1.]],
 
         [[1.]]],
 
 
        [[[1.]],
 
         [[1.]],
 
         [[1.]],
 
         ...,
 
         [[1.]],
 
         [[1.]],
 
         [[1.]]],
 
 
        [[[1.]],
 
         [[1.]],
 
         [[1.]],
 
         ...,
 
         [[1.]],
 
         [[1.]],
 
         [[1.]]]], dtype=float32)>]

In [6]:
tf.__version__

'2.15.0'

In [7]:
tfp.__version__

'0.23.0'

In [3]:
keras.__version__

'2.15.0'

In [6]:

@tf.function(reduce_retracing=True)
def get_grad(x_TF):
    # number of score function trick samples
    N=5
    model_distribution = model(x_TF)
    sample_y_NT = model_distribution.sample(N)
    with tf.GradientTape() as tape:
        
        model_distribution = model(x_TF)
        #stopped_samples = tf.stop_gradient(sample_y_NT)
        sample_log_probs_NT = model_distribution.log_prob(sample_y_NT)

    # use jacobian to get gradient of each sample
    jacobian_params_NT = tape.jacobian(sample_log_probs_NT, model.trainable_weights)
    return jacobian_params_NT

In [7]:
get_grad(x_TF)